# Center of Mass

### **Introduction**

The center of mass (COM) is a fundamental concept in physics that refers to the average position of all the mass in a system. It is the point at which the total mass of an object or system can be considered to be concentrated for the purpose of analyzing motion. In simple terms, it's the balance point of an object. The center of mass plays a critical role in understanding how objects move, rotate, and interact under various forces.

### **What is it?**

The center of mass is the weighted average position of all the mass in an object or system. Mathematically, it is calculated by considering the masses of individual components and their positions relative to a chosen origin.

For a continuous mass distribution, this becomes an integral.

The center of mass is particularly important in analyzing the motion of objects, especially when there are rotational forces or external interactions involved. It allows for simplifications in equations of motion, as the motion of the entire system can be described as though all mass is concentrated at a single point, the center of mass.

### **Example**

Consider a simple system, like a uniform rod with length $L$ and mass $M$. To find the center of mass, we can use the fact that for a uniform object, the center of mass is located at the midpoint of the object.

For the rod:

- The mass is uniformly distributed along the length of the rod.
- Therefore, the center of mass is at the point $L/2$ from either end of the rod.

In this case, the center of mass simplifies to the midpoint of the rod because of its uniform mass distribution.

# Simulation

## Setup

Installing packages (for Google Colab). If this notebook is opened in Google Colab then some packages must be installed to run the code!\
Then importing the scene and plot settings.

In [ ]:
#@title Run to install MuJoCo and `dm_control` for Google Colab

IS_COLAB = 'google.colab' in str(get_ipython())
if IS_COLAB:
    # download the repository
    !git clone https://github.com/commanderxa/alphalabs.git
    from alphalabs.mechanics.setup import install_packages_colab
    install_packages_colab()
    import alphalabs.mechanics.plot
    from alphalabs.mechanics.scene import Scene
else:
    import os, sys
    module_path = os.path.abspath(os.path.join(".."))
    if module_path not in sys.path:
        sys.path.append(module_path)
    # import the scene
    from scene import Scene
    import plot

## Import

Import all required packages to preform simulations. Packages include simulation engine, plotting libraries and other ones necessary for computations.

In [ ]:
%env MUJOCO_GL=egl

import os

# simulation
from dm_control import mujoco, mjcf
from dm_control.mujoco.wrapper.mjbindings import enums

# for video recording
import mediapy

# computations
import numpy as np

## Initial Conditions

In this block constants are defined. They impact the environment, rendering and objects directly.

In [ ]:
# global
viscosity = 0.00002  # Air Resistance

# simulation constants
masses = [0.1, 20, 3, 0.6, 0.3, 1, 8]
region = (np.array([-2.0, -2.0, 0.0]), np.array([2.0, 2.0, 4.0]))

# rendering
width = 1920
height = 1080
dpi = 600
duration = 10  # (seconds)
framerate = 60  # (Hz)

## Model

### World Model

Collecting everything into one general model

In [ ]:
class Model(object):

    def __init__(
        self, masses: list[float], region: tuple[np.ndarray, np.ndarray]
    ) -> None:
        self.model = mjcf.RootElement(model="center_of_mass")

        # set render info
        self.model.visual.__getattr__("global").offheight = height
        self.model.visual.__getattr__("global").offwidth = width

        self.model.option.viscosity = viscosity

        # create the scene (ground)
        self.scene = Scene(
            length=(region[1][0] - region[0][0]) * 2,
            width=(region[1][1] - region[0][1]) * 2,reflectance=0
        )
        self.scene_site = self.model.worldbody.add("site", pos=[0, 0, 0])
        self.scene_site.attach(self.scene.model)

        # create the group of objects with mass
        self.scene = self.model.worldbody.add("body", name="sphere_group", pos=[0, 0, 0])
        # generate the spheres
        for mass in masses:
            sphere_pos = np.random.uniform(region[0], region[1])
            self.scene.add(
                "geom", name=f"{mass} kg", mass=mass, pos=sphere_pos, size=[0.2]
            )

## Simulation

Initializing the `physics` of the simulation

In [ ]:
model = Model(masses, region).model
physics = mjcf.Physics.from_mjcf_model(model)

First of all, the environment must be verified by rendering a picture

In [ ]:
scene_option = mujoco.wrapper.core.MjvOption()
scene_option.label = enums.mjtLabel.mjLABEL_GEOM  # enable body labels

mediapy.show_image(
    physics.render(height=height, width=width, scene_option=scene_option)
)

Save the rendered image

In [ ]:
image_name = f"com_not_shown" if IS_COLAB else f"../../output/com_not_shown"
mediapy.write_image(image_name + ".png", image=physics.render(height=height, width=width, scene_option=scene_option))

Now we enable `MuJoCo` to show the center of mass (COM). However, we will find that COM is shown as white spheres. There are 3 spheres since COM is shown for each body and for the entire world, which is nice. The problem is in the size or color of COM spheres, they may be near each other, which makes it a bit uncomfortable to look at them.

In [ ]:
scene_option.flags[enums.mjtVisFlag.mjVIS_COM] = True
mediapy.show_image(physics.render(height=height, width=width, scene_option=scene_option))

We can add our own COM spheres. First of all, we must get the coordintaes of the COM.

In [ ]:
physics.named.data.subtree_com

Now, let's take only the interesting coordinates.

In [ ]:
physics.named.data.subtree_com["sphere_group"]

Let's add a big red sphere at the location of these coordinates.

In [ ]:
model.worldbody.add(
    "geom", name="center of mass", pos=physics.named.data.subtree_com["sphere_group"], size=[0.5], rgba=[1, 0, 0, 1]
)

Update the physics since we updated the model.

In [ ]:
physics = mjcf.Physics.from_mjcf_model(model)

Render the image again, and voilà!

In [ ]:
scene_option.flags[enums.mjtVisFlag.mjVIS_COM] = False
scene_option.flags[enums.mjtLabel.mjLABEL_GEOM] = True
mediapy.show_image(physics.render(height=height, width=width, scene_option=scene_option))

Saving the new rendered image

In [ ]:
image_name = f"com_shown" if IS_COLAB else f"../../output/com_shown"
mediapy.write_image(image_name + ".png", image=physics.render(height=height, width=width, scene_option=scene_option))